In [1]:
import warnings
warnings.filterwarnings('ignore')

# Install required packages if not already installed
try:
    import yfinance as yf
except ImportError:
    !pip install yfinance
    import yfinance as yf


In [2]:
!pip install yfinance --quiet

import yfinance as yf
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

device = "cuda" if torch.cuda.is_available() else "cpu"
device


'cpu'

In [3]:
import datetime as dt
import requests
import math
import time
import io

In [4]:
myTickers21 = ['BCC','BKNG','BLDR','BRK-B','CAAP','CACI','CEG','CMPR','CNM','CPRT','CW','CXT','EXTR','FICO',
              'FOR','FTV','GE','GFS','GNTX','GOOGL','HUBB','HWM','INTU','JPM','LLY','MCK','META','MMYT',
              'MOD','NEU','NFLX','NVDA','PRGS','PRIM','SCHW','STRL','GTLB','TEX','TS','TT','UBER',
              'URI','VECO','VRSK','VRT','WFRD','WMS','WMT','ZS','COCO','AGM','CAVA','ETN','EME']


etf_tickers = ['SPY','QQQ','IWM','SMH','IGV',
                'XLB','XLE','XLF','XLI','XLK','XLP','XLU','XLV','XLY','XLC','XLRE','XOP','XES','XME','XBI',
                'ITA','ITB','IAI','IHI',
                'FAN','TAN','DIA',
                'GLD','GDX','SLV','DBC','DBB','DBO','UNG']

levetf_tickers = ['TQQQ','UPRO','UDOW','URTY','SOXL','LABU','FAS','ERX','UCO','SVXY','UVXY','BOIL','UGL','AGQ']



# Get DJ30 Tickers
DJ30_tickers = ['AAPL', 'AMGN', 'AMZN','AXP','BA', 'CAT', 'CRM','CSCO', 'CVX', 'DIS', 'DOW', 'GS', 'HD','HON',
           'IBM', 'INTC', 'JNJ', 'JPM', 'KO', 'MCD', 'MMM', 'MRK', 'MSFT', 'NKE',
           'PFE', 'PG', 'TRV', 'UNH', 'V', 'VZ', 'WMT', 'XOM']


# Use headers to avoid 403 error
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

# Read NASDAQ-100 tickers from Wikipedia with headers
url = 'https://en.wikipedia.org/wiki/Nasdaq-100'
response = requests.get(url, headers=headers)
tables = pd.read_html(response.content)

# The constituents table is typically the 4th or 5th table
# Let's find the right one by checking for 'Ticker' column
for i, table in enumerate(tables):
    if 'Ticker' in table.columns:
        nasdaq100_df = table
        print(f"Found ticker table at index {i}")
        break

# Extract ticker symbols and convert to list
nasdaq_tickers = nasdaq100_df['Ticker'].tolist()

# Read S&P 500 tickers from Wikipedia
sp500_url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
sp500_response = requests.get(sp500_url, headers=headers)
sp500_tables = pd.read_html(sp500_response.content)

# First table contains S&P 500 constituents
sp500_df = sp500_tables[0]

# Extract ticker symbols and convert to list
sp500_tickers = sp500_df['Symbol'].tolist()

merged_tickers = DJ30_tickers + nasdaq_tickers + sp500_tickers + myTickers21 + etf_tickers
tickers = merged_tickers
tickers=nasdaq_tickers + ["SPY","QQQ","DIA","SVXY","GLD","USO","SMH","IGV","SPMO","VOOG","VOOV","USMV","SPLV","SPHB","SCHD","VWO","FEZ","FAS"]
#tickers = nasdaq_tickers
#tickers = ["SPY","FAS","DIA","SVXY","GLD","USO","SMH","IGV","SPMO","VOOG","VOOV","USMV","SPLV","SPHB","SCHD","VWO","FEZ","QQQ"]
#tickers = ["SPY","DIA","QQQ","LRCX","UVXY"]
#tickers = ["SPY","QQQ","DIA","SVXY","GLD","USO","SMH","IGV","SPMO","VOOG","VOOV","USMV","SPLV","SPHB","SCHD","VWO","FEZ","FAS"]
tickers = nasdaq_tickers + ["SPY","QQQ","DIA","SVXY","UVXY","GLD","USO","SMH","IGV","SPMO"]
to_remove = {"CHTR", "KDP", "MELI","NXPI","TSLA","VRSK"}
tickers = [t for t in tickers if t not in to_remove]
tickers = DJ30_tickers



Found ticker table at index 4


In [5]:
#tickers = ["SPY", "QQQ", "SMH", "IGV", "GLD", "SVXY"]
#tickers = ["SPY", "NVDA", "ASML", "AVGO", "INTC", "AMD","IBM","GOOGL","AMZN","MSFT","SOXL","SMH","QQQ","TSM","ALAB","IGV","GLD","CEG"]
data = yf.download(tickers, start="2025-01-01", end="2025-12-26")
#data.tail()


[*********************100%***********************]  32 of 32 completed


In [6]:
# Case 1: MultiIndex
if isinstance(data.columns, pd.MultiIndex):
    # Use close if available, otherwise adj close
    price_key = "Adj Close" if "Adj Close" in data.columns.levels[0] else "Close"
    volume_key = "Volume"

    prices = data[price_key][tickers]
    volume = data[volume_key][tickers]

# Case 2: Single flat index
else:
    # Try any possible form of close price
    prices = pd.DataFrame(index=data.index)
    volume = pd.DataFrame(index=data.index)

    for t in tickers:
        candidates = [
            f"Close_{t}",
            f"Adj Close_{t}",
            f"{t}",
            f"{t}_Close",
            f"{t}_Adj Close",
        ]

        # Find the first column that exists
        found_price = None
        for c in candidates:
            if c in data.columns:
                found_price = c
                break
        if found_price is None:
            print(f"WARNING: No price data found for {t}, filling with NaN.")
            prices[t] = np.nan
        else:
            prices[t] = data[found_price]

        # Volume extraction
        vol_candidates = [
            f"Volume_{t}",
            f"{t}_Volume",
            f"{t}_Vol",
        ]

        found_vol = None
        for c in vol_candidates:
            if c in data.columns:
                found_vol = c
                break

        if found_vol is None:
            print(f"WARNING: No volume found for {t}, filling with NaN.")
            volume[t] = np.nan
        else:
            volume[t] = data[found_vol]


In [7]:
valid = prices.notna().all()
prices = prices.loc[:, valid]
volume = volume.loc[:, valid]

print("Using tickers:", list(prices.columns))


Using tickers: ['AAPL', 'AMGN', 'AMZN', 'AXP', 'BA', 'CAT', 'CRM', 'CSCO', 'CVX', 'DIS', 'DOW', 'GS', 'HD', 'HON', 'IBM', 'INTC', 'JNJ', 'JPM', 'KO', 'MCD', 'MMM', 'MRK', 'MSFT', 'NKE', 'PFE', 'PG', 'TRV', 'UNH', 'V', 'VZ', 'WMT', 'XOM']


In [8]:
def RSI(series, period=14):
    delta = series.diff()
    up = delta.clip(lower=0)
    down = -1 * delta.clip(upper=0)
    ma_up = up.rolling(period).mean()
    ma_down = down.rolling(period).mean()
    rs = ma_up / ma_down
    return 100 - (100 / (1 + rs))

def zscore(x, window=20):
    return (x - x.rolling(window).mean()) / x.rolling(window).std()


In [9]:
features = {}

for t in tickers:
    p = prices[t]
    v = volume[t]

    df = pd.DataFrame()
    df["return_1d"] = p.pct_change()
    df["return_5d"] = p.pct_change(5)
    df["return_21d"] = p.pct_change(21)
    df["vol_21d"] = p.pct_change().rolling(21).std()
    df["ma10"] = p.rolling(10).mean()
    df["ma20"] = p.rolling(20).mean()
    df["ma50"] = p.rolling(50).mean()
    df["price_ma10"] = p / df["ma10"]
    df["price_ma20"] = p / df["ma20"]
    df["price_ma50"] = p / df["ma50"]
    df["rsi14"] = RSI(p, 14)
    df["vol_z"] = zscore(v)

    features[t] = df


In [10]:
# Combine into a single panel: [time, asset, feature]
panel = np.stack([features[t].values for t in tickers], axis=1)
dates = features[tickers[0]].index

panel.shape, len(dates)


((246, 32, 12), 246)

In [11]:
future_returns = prices.pct_change().shift(-1).loc[dates].values


In [12]:
min_len = min(panel.shape[0], future_returns.shape[0])
X = panel[:min_len]
Y = future_returns[:min_len]


In [13]:
mask = ~np.isnan(X).any(axis=(1,2)) & ~np.isnan(Y).any(axis=1)
X = X[mask]
Y = Y[mask]


In [14]:
X_t = torch.tensor(X, dtype=torch.float32).to(device)
Y_t = torch.tensor(Y, dtype=torch.float32).to(device)

X_t.shape, Y_t.shape


(torch.Size([196, 32, 12]), torch.Size([196, 32]))

In [15]:
num_assets = len(tickers)
num_features = X_t.shape[2]
num_regimes = 3   # bull/bear/neutral

class HierarchicalFinanceHRM(nn.Module):
    def __init__(self, num_assets, num_features, num_regimes, hidden=32):
        super().__init__()

        # Level 1: Per-asset encoder
        self.asset_encoder = nn.Sequential(
            nn.Linear(num_features, hidden),
            nn.ReLU(),
            nn.Linear(hidden, hidden)
        )

        # Level 2: Regime predictor
        self.regime_net = nn.Sequential(
            nn.Linear(num_assets * hidden, hidden),
            nn.ReLU(),
            nn.Linear(hidden, num_regimes)
        )

        # Regime embeddings
        self.regime_embed = nn.Embedding(num_regimes, hidden)

        # Level 3: Portfolio allocator
        self.alloc_net = nn.Sequential(
            nn.Linear(hidden + hidden, hidden),
            nn.ReLU(),
            nn.Linear(hidden, num_assets),
            nn.Softmax(dim=-1)
        )

    def forward(self, X):
        # X: [batch, assets, features]
        asset_vecs = self.asset_encoder(X)

        flat = asset_vecs.reshape(X.size(0), -1)
        regime_logits = self.regime_net(flat)
        regime_probs = torch.softmax(regime_logits, dim=-1)

        regime_emb = regime_probs @ self.regime_embed.weight
        asset_mean = asset_vecs.mean(dim=1)

        combined = torch.cat([regime_emb, asset_mean], dim=-1)
        weights = self.alloc_net(combined)

        return regime_logits, weights


In [16]:
def train_once(X_t, Y_t, num_assets, num_features, num_regimes, epochs=200):

    # ----- BUILD MODEL -----
    model = HierarchicalFinanceHRM(num_assets, num_features, num_regimes).to(device)

    optimizer = optim.Adam(model.parameters(), lr=1e-4)

    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer,
        mode='min',     # because you minimize loss
        factor=0.7,
        patience=5
    )

    loss_ce = nn.CrossEntropyLoss()

    # ----- BUILD REGIME LABELS -----
    mean_returns_np = Y_t.mean(axis=1).cpu().numpy()
    regime_label = torch.tensor(
        pd.qcut(mean_returns_np, q=3, labels=False),
        dtype=torch.long
    ).to(device)

    lambda_profit = 0.5   # kept from original, although unused now

    # ----- TRAINING LOOP -----
    for epoch in range(epochs):
        optimizer.zero_grad()

        regime_logits, weights = model(X_t)

        # regime classification loss
        loss_reg = loss_ce(regime_logits, regime_label)

        # compute portfolio return series
        port_ret_series = (weights * Y_t).sum(axis=1)

        mean_ret = port_ret_series.mean()
        vol      = port_ret_series.std()

        sharpe = mean_ret / (vol + 1e-6)

        entropy = -(weights * weights.log()).sum(dim=1).mean()

        # downside deviation
        downside = port_ret_series[port_ret_series < 0].std()

        # final loss (your last chosen objective)
        loss = -mean_ret / (downside + 1e-6)

        # backward + update
        loss.backward()
        optimizer.step()

        # scheduler step
        scheduler.step(loss.item())

        # optional logging
        if epoch % 10 == 0:
            current_lr = optimizer.param_groups[0]['lr']
            print(f"Epoch {epoch:3d} | LR={current_lr:.6f} | Loss={loss:.6f}")

    # ----- RETURN FINAL WEIGHTS -----
    with torch.no_grad():
        _, w = model(X_t[-1].unsqueeze(0))
        return w.cpu().numpy().flatten()


In [17]:
num_runs = 50
all_weights = []

for i in range(num_runs):
    print(f"\n----- RUN {i+1} -----")
    w = train_once(X_t, Y_t, num_assets, num_features, num_regimes, epochs=200)
    all_weights.append(w)

all_weights = np.array(all_weights)
avg_weights = all_weights.mean(axis=0)



----- RUN 1 -----
Epoch   0 | LR=0.000100 | Loss=-0.069182
Epoch  10 | LR=0.000100 | Loss=-0.110834
Epoch  20 | LR=0.000100 | Loss=-0.154242
Epoch  30 | LR=0.000100 | Loss=-0.157002
Epoch  40 | LR=0.000100 | Loss=-0.157645
Epoch  50 | LR=0.000100 | Loss=-0.159011
Epoch  60 | LR=0.000100 | Loss=-0.163672
Epoch  70 | LR=0.000100 | Loss=-0.191347
Epoch  80 | LR=0.000100 | Loss=-0.211794
Epoch  90 | LR=0.000100 | Loss=-0.241618
Epoch 100 | LR=0.000100 | Loss=-0.253213
Epoch 110 | LR=0.000100 | Loss=-0.265957
Epoch 120 | LR=0.000100 | Loss=-0.277641
Epoch 130 | LR=0.000100 | Loss=-0.287565
Epoch 140 | LR=0.000100 | Loss=-0.296378
Epoch 150 | LR=0.000100 | Loss=-0.305777
Epoch 160 | LR=0.000100 | Loss=-0.314252
Epoch 170 | LR=0.000100 | Loss=-0.322363
Epoch 180 | LR=0.000100 | Loss=-0.330072
Epoch 190 | LR=0.000100 | Loss=-0.337642

----- RUN 2 -----
Epoch   0 | LR=0.000100 | Loss=-0.072271
Epoch  10 | LR=0.000100 | Loss=-0.105533
Epoch  20 | LR=0.000100 | Loss=-0.143844
Epoch  30 | LR=0.00

In [18]:
print("\n=== Weights From Each Run (Sorted) ===")
for i, w in enumerate(all_weights):
    print(f"\nRun {i+1} (sorted):")
    tw_sorted_run = sorted(zip(tickers, w), key=lambda x: x[1], reverse=True)
    for t, weight in tw_sorted_run:
        print(f"  {t}: {weight:.3f}")



=== Weights From Each Run (Sorted) ===

Run 1 (sorted):
  GS: 0.714
  WMT: 0.285
  KO: 0.001
  XOM: 0.000
  DIS: 0.000
  AXP: 0.000
  HON: 0.000
  AMZN: 0.000
  AAPL: 0.000
  CRM: 0.000
  CAT: 0.000
  BA: 0.000
  PG: 0.000
  HD: 0.000
  DOW: 0.000
  MCD: 0.000
  MMM: 0.000
  JPM: 0.000
  PFE: 0.000
  IBM: 0.000
  UNH: 0.000
  TRV: 0.000
  NKE: 0.000
  V: 0.000
  AMGN: 0.000
  INTC: 0.000
  CSCO: 0.000
  MSFT: 0.000
  MRK: 0.000
  JNJ: 0.000
  CVX: 0.000
  VZ: 0.000

Run 2 (sorted):
  JNJ: 0.471
  AXP: 0.366
  JPM: 0.100
  INTC: 0.035
  IBM: 0.011
  BA: 0.007
  DIS: 0.003
  KO: 0.001
  VZ: 0.001
  PG: 0.001
  NKE: 0.001
  CVX: 0.001
  TRV: 0.001
  MSFT: 0.000
  V: 0.000
  MRK: 0.000
  MCD: 0.000
  WMT: 0.000
  AAPL: 0.000
  CSCO: 0.000
  MMM: 0.000
  GS: 0.000
  AMGN: 0.000
  AMZN: 0.000
  CRM: 0.000
  HON: 0.000
  CAT: 0.000
  XOM: 0.000
  HD: 0.000
  DOW: 0.000
  PFE: 0.000
  UNH: 0.000

Run 3 (sorted):
  CAT: 0.358
  JNJ: 0.339
  WMT: 0.299
  MCD: 0.001
  VZ: 0.001
  JPM: 0.001
  HO

In [19]:
tw_sorted = sorted(zip(tickers, avg_weights), key=lambda x: x[1], reverse=True)

print("\n=== Ensemble-Averaged Portfolio Weights ===")
for t, w in tw_sorted:
    print(f"{t}: {w:.3f}")



=== Ensemble-Averaged Portfolio Weights ===
CAT: 0.174
WMT: 0.141
JNJ: 0.115
GS: 0.114
JPM: 0.075
MSFT: 0.071
INTC: 0.071
DIS: 0.044
AXP: 0.042
BA: 0.037
CSCO: 0.032
IBM: 0.028
AMGN: 0.015
TRV: 0.013
MRK: 0.009
HD: 0.009
MCD: 0.004
KO: 0.002
V: 0.001
MMM: 0.000
AAPL: 0.000
VZ: 0.000
NKE: 0.000
HON: 0.000
PG: 0.000
CRM: 0.000
AMZN: 0.000
UNH: 0.000
PFE: 0.000
CVX: 0.000
XOM: 0.000
DOW: 0.000
